# Fit $G(t)$

\begin{equation}
G(t) = G_N^0 \int \frac{h\left(\tau\right)}{\tau} e^{-\frac{t}{\tau}} \mathrm{d}\tau \\
\end{equation}

If we don't know the form of relaxation spectrum a priori, we can start with the discrete spectrum
\begin{equation}
h\left(\tau\right) = \sum_i g_i \delta(\tau - \tau_i)
\end{equation}

\begin{equation}
G(t) = G_N^0 \int \frac{h\left(\tau\right)}{\tau} e^{-\frac{t}{\tau}} \mathrm{d}\tau = G_N^0 \sum_i \frac{g_i}{\tau_i} e^{-\frac{t}{\tau_i}}
\end{equation}

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares, minimize
from scipy.misc import logsumexp

from mpmath import *
mp.dps = 25; mp.pretty = True
from scipy.optimize import broyden1

In [ ]:
#Define BSW functions: G(t), G*(omega), h(tau)
def supp_prod(tauv, alpha, i):
    result = 1
    for j in range (1, i+1):
        result *= tauv[j]**(alpha[j - 1] - alpha[j])
    
    return result

def Gs_BSW(frequency, alpha, tauv, n, G0):
    result = []
    for omega in frequency: 
        sum1 = 0
        for i in range(0, n):
            sum1 += gamma(1 + alpha[i])*(tauv[i + 1]**(1 + alpha[i])*hyp2f1(1, 1 + alpha[i], 2 + alpha[i], -j*tauv[i + 1]*omega)/gamma(2 + alpha[i]) - tauv[i]**(1 + alpha[i])*hyp2f1(1, 1 + alpha[i], 2 + alpha[i], -j*tauv[i]*omega)/gamma(2 + alpha[i]))*supp_prod(tauv, alpha, i)
        sum2 = 0
        for i in range(0, n):
            sum2 += (tauv[i + 1]**alpha[i] - tauv[i]**alpha[i])/alpha[i]*supp_prod(tauv, alpha, i)
        res =j*omega*sum1/sum2*G0
        result.append((re(result),im(result)))
    
    return result

def Gt_BSW(time, alpha, tauv, n, G0):
    result = []
    for t in time: 
        sum1 = 0
        for i in range(0, n):
            sum1 += t**alpha[i] * ( gammainc(-alpha[i],t/tauv[i+1]) - gammainc(-alpha[i],t/tauv[i]) )*supp_prod(tauv, alpha, i)
        sum2 = 0
        for i in range(0, n):
            sum2 += (tauv[i + 1]**alpha[i] - tauv[i]**alpha[i])/alpha[i]*supp_prod(tauv, alpha, i)
        result.append(sum1/sum2*G0)
    
    return result

def h_BSW(time, alpha, tauv, n):
    result = []
    for t in time: 
        sum1 = 0
        for i in range(0, n):
            sum1 += t**alpha[i] * np.heaviside(tauv[i+1]-t,0) * np.heaviside(t-tauv[i],0) * supp_prod(tauv, alpha, i)
        sum2 = 0
        for i in range(0, n):
            sum2 += (tauv[i + 1]**alpha[i] - tauv[i]**alpha[i])/alpha[i]*supp_prod(tauv, alpha, i)
        result.append(sum1/sum2)
    
    return result

#Define BSW fits for linear chains based on paper 'Analytic slip-link expressions for universal dynamic modulus predictions of linear monodisperse polymer melts'
def Gs_CFSM(omega, Nc):
    n=3
    alpha1 = (-0.0005, -0.0205)
    alpha2 = (0.00029, 0.109957)
    alpha3 = (17.69589, 1.04026, -0.00095677)
    tau1 = (0.6288876, 0.119458)
    tau2 = (1.52508156, 0.02996758796795)
    tau3 = (3.110954, 0.022615)
    tau4 = (3.4840295, 0.0142809)
    alpha = (alpha1[0]*Nc + alpha1[1], alpha2[0]*Nc + alpha2[1], alpha3[0]/Nc + alpha3[1] + alpha3[2]*Nc)
    tauv = (tau1[1]*Nc**tau1[0], tau2[1]*Nc**tau2[0], tau3[1]*Nc**tau3[0], tau4[1]*Nc**tau4[0])
    return Gs_BSW(omega, alpha, tauv, n, 1)

def Gt_CFSM(t, Nc):
    n=3
    alpha1 = (-0.0005, -0.0205)
    alpha2 = (0.00029, 0.109957)
    alpha3 = (17.69589, 1.04026, -0.00095677)
    tau1 = (0.6288876, 0.119458)
    tau2 = (1.52508156, 0.02996758796795)
    tau3 = (3.110954, 0.022615)
    tau4 = (3.4840295, 0.0142809)
    alpha = (alpha1[0]*Nc + alpha1[1], alpha2[0]*Nc + alpha2[1], alpha3[0]/Nc + alpha3[1] + alpha3[2]*Nc)
    tauv = (tau1[1]*Nc**tau1[0], tau2[1]*Nc**tau2[0], tau3[1]*Nc**tau3[0], tau4[1]*Nc**tau4[0])
    return Gt_BSW(t, alpha, tauv, n, 1)

def h_CFSM(t, Nc):
    n=3
    alpha1 = (-0.0005, -0.0205)
    alpha2 = (0.00029, 0.109957)
    alpha3 = (17.69589, 1.04026, -0.00095677)
    tau1 = (0.6288876, 0.119458)
    tau2 = (1.52508156, 0.02996758796795)
    tau3 = (3.110954, 0.022615)
    tau4 = (3.4840295, 0.0142809)
    alpha = (alpha1[0]*Nc + alpha1[1], alpha2[0]*Nc + alpha2[1], alpha3[0]/Nc + alpha3[1] + alpha3[2]*Nc)
    tauv = (tau1[1]*Nc**tau1[0], tau2[1]*Nc**tau2[0], tau3[1]*Nc**tau3[0], tau4[1]*Nc**tau4[0])
    return h_BSW(t, alpha, tauv, n)

def Gs_R(omega):
    n=3
    alpha = (0.64635, -0.4959, -1.2716)
    tauv = (6.313268381616272e-9, 2.181509372282138e-7, 0.797317365925168,  18.201382525250114)
    GR = 1942.29;
    return Gs_BSW(omega, alpha, tauv, n, GR)

def Gt_MMM(time, params):
    #Variable frequencies
    lambdaArr = np.split(params,2)[0]
    gArr = np.split(params,2)[1]/np.sum(np.split(params,2)[1])
    #Fixed frequencies
    #lambdaArr=10.0**((np.array(range(nmodes), float) + 1.0)/nmodes*np.log10(tfinal))
    #gArr = params/np.sum(params)
    return np.dot(np.exp(-time/lambdaArr), gArr)*GN0

def log_Gt_MMM(time,params):
    lambdaArr = np.split(params,2)[0]
    gArr = np.split(params,2)[1]/np.sum(np.split(params,2)[1])
    #lambdaArr=10.0**((np.array(range(nmodes), float) + 1.0)/nmodes*np.log10(tfinal))
    #gArr = params/np.sum(params)
    return logsumexp(-time/lambdaArr, b=gArr*GN0)

#Vectorize function fdt and log_fdt
Gt_MMM_vec=np.vectorize(Gt_MMM, excluded=['params'])
log_Gt_MMM_vec=np.vectorize(log_Gt_MMM, excluded=['params'])

#Define residuals
def residuals_Gt_MMM(param):
    return Gt_MMM_vec(time=x, params=param)-y

def residuals_log_Gt_MMM(param):
    if np.any(Gt_MMM_vec(time=x[:-1], params=param) < 0):
        return np.full(x[:-1].shape,1e8) #Penalty for negative f_d(t)
    else:
        return log_Gt_MMM_vec(time=x, params=param)-np.log(y)

def MSE_MMM(param):
    return np.dot(residuals_Gt_MMM(param),residuals_Gt_MMM(param))/np.size(x)

def log_MSE_MMM(param):
    return np.dot(residuals_log_Gt_MMM(param),residuals_log_Gt_MMM(param))/np.size(x)

def Gp_MMM(omega, params):
    lambdaArr = np.split(params,2)[0]
    gArr = np.split(params,2)[1]/np.sum(np.split(params,2)[1])
    
    return np.sum((gArr * lambdaArr**2 * omega**2)/(1 + lambdaArr**2 * omega**2))*GN0

def Gdp_MMM(omega, params):
    lambdaArr = np.split(params,2)[0]
    gArr = np.split(params,2)[1]/np.sum(np.split(params,2)[1])
    
    return np.sum((gArr * lambdaArr * omega)/(1 + lambdaArr**2 * omega**2))*GN0

#Vectorize function Gp and Gdp
Gp_MMM_vec=np.vectorize(Gp_MMM, excluded=['params'])
Gdp_MMM_vec=np.vectorize(Gdp_MMM, excluded=['params'])

**First, generate $h(\tau)$ and $G(t)$ for $N_c=257$**

In [ ]:
npoints=1000
x=10**(1+(np.array(range(npoints), float) + 1.0)/npoints*5)
y=h_CFSM(x, 257)

fig = plt.figure(figsize=(8, 6))

ax1 = fig.add_subplot(111)

ax1.set_title("Entanglement lifetime distribution")
ax1.set_xlabel(r'$\tau/\tau_c$')
ax1.set_ylabel(r'$h(\tau)$')
plt.xlim(xmin=1, xmax=1e6)
ax1.plot(x,y, c='r', label=r'$h(\tau)$')

leg = ax1.legend()
ax1.set_xscale('log')
ax1.set_yscale('log')

plt.show()

In [ ]:
GN0=(257-3)/2.0

npoints=100
x=10**(0+(np.array(range(npoints), float) + 1.0)/npoints*7)
y=np.array(Gt_CFSM(x, 257), dtype='float')*GN0

tfinal=x[-1]


fig = plt.figure(figsize=(8, 6))

ax1 = fig.add_subplot(111)

ax1.set_title("Entanglement lifetime distribution")
ax1.set_xlabel(r'$t/\tau_c$')
ax1.set_ylabel(r'$G(t)$')
plt.xlim(xmin=1, xmax=1e7)
ax1.scatter(x,y, c='r', label=r'$G(t)$')

leg = ax1.legend()
ax1.set_xscale('log')
ax1.set_yscale('log')

plt.show()

## Optimizing strategy:
1. Start with some big number of modes, like $n_{modes}=10$
2. Run the least square optimization with standard residuals $y_i-f(x_i)$.
3. Scan through different number of modes down to 1.
4. If fit results have any negative weights, $g_i<0$, ignore
5. If fit results have all positive weights, run the least square optimization with log-residuals
6. Choose the best result among step 5

** Use SciPy numeric least square minimization algorithm **

## 1. Fit G(t) with Multi-Mode Maxwell expressions. Optimize using standard residuals $G_i-G^{MMM}(t_i)$. Allow both $g_i$ and $\lambda_i$ to be adjusted

In [ ]:
fits_1 = [] #output of fitting function for all tested numbers of modes
#successful_fits_1 = [] #number of modes for successful fits
#for nmodes in range(1, 15):
nmodes = 10
lambdaArrInit=10.0**((np.array(range(nmodes), float) + 1.0)/nmodes*np.log10(tfinal))
gArrInit=np.full(nmodes, 1.0/nmodes)

#fit = least_squares(residuals_gt, gArrInit, xtol=1e-15)
fit = least_squares(residuals_Gt_MMM, np.append(lambdaArrInit, gArrInit), xtol=1e-15)
fit.x
#fits_1.append(fit)
#if fit.success and not np.any(fdtvec(time=x, params=fit.x) < 0):
#    successful_fits_1.append(nmodes)

In [ ]:
fig1 = plt.figure(figsize=(24, 6))

ax0 = fig1.add_subplot(131)

ax0.set_title(r'$h\left(\tau\right)/\tau$')
ax0.set_xlabel(r'$\lambda$')
ax0.set_ylabel(r'$g$')

ax0.scatter(lambdaArrInit,gArrInit, c='k', label=r'Initial guess')
ax0.scatter(np.split(fit.x,2)[0], np.split(fit.x,2)[1]/np.sum(np.split(fit.x,2)[1]), c='r', label=r'First fit')
leg = ax0.legend()
ax0.set_xscale('log')

ax1 = fig1.add_subplot(132)

ax1.set_title("Check results of the fit")
ax1.set_xlabel(r'$t/\tau_c$')
ax1.set_ylabel(r'log residuals')

ax1.plot(x,Gt_MMM_vec(time=x, params=np.append(lambdaArrInit, gArrInit)), c='k', label=r'Initial guess')
ax1.plot(x,Gt_MMM_vec(time=x, params=fit.x), c='r', label=r'First fit')
ax1.plot(x,y, c='g', label=r'$f_d(t)$')

leg = ax1.legend()
ax1.set_xscale('log')
ax1.set_yscale('log')

ax2 = fig1.add_subplot(133)

ax2.set_title(r'$f_d(t)$')
ax2.set_xlabel(r'$t/\tau_c$')
ax2.set_ylabel(r'$f_d(t)$')

ax2.plot(x,Gt_MMM_vec(time=x, params=np.append(lambdaArrInit, gArrInit)), c='k', label=r'Initial guess')
ax2.plot(x,Gt_MMM_vec(time=x, params=fit.x), c='r', label=r'First fit')
ax2.plot(x,y, c='g', label=r'Simulation data')
leg = ax2.legend()
ax2.set_xscale('log')

plt.show()

## 2. Improve fit using log-residuals $\log(G_i)-\log(G^{MMM}(t_i))$ to find best fit for the longest relaxation tail 

In [ ]:
fits_2 = [] #output of fitting function for all tested numbers of modes
# best_nmodes = successful_fits_1[0]
# for i in successful_fits_1:
#     fit = fits_1[i-1]
#print('nmodes\t{0}'.format(i))
print(fit.message)
print('Initial guess MSE\t{0}'.format(MSE_MMM(gArrInit)))
print('Fit MSE\t\t\t{0}'.format(MSE_MMM(fit.x)))

fit2 = least_squares(residuals_log_Gt_MMM, fit.x, xtol=1e-14, ftol=1e-14)
fits_2.append(fit2)

if fit2.success:
    print(fit2.message)
    print('First fit log-MSE\t{0}'.format(log_MSE_MMM(fit.x)))
    print('Second fit log-MSE\t{0}'.format(log_MSE_MMM(fit2.x)))

print(' ')

In [ ]:
fit2.x

In [ ]:
# fit = fits_1[8-1]
# fit2 = best_fit
h=h_CFSM(x, 257)

fig3 = plt.figure(figsize=(24, 6))

ax0 = fig3.add_subplot(131)

ax0.set_title(r'$p^{cr}\left(\tau\right)$')
ax0.set_xlabel(r'$\lambda$')
ax0.set_ylabel(r'$g$')

ax0.scatter(lambdaArrInit,gArrInit, c='k', label=r'Initial guess')
ax0.scatter(np.split(fit.x,2)[0], np.split(fit.x,2)[1]/np.sum(np.split(fit.x,2)[1]), c='r', label=r'First fit')
ax0.scatter(np.split(fit2.x,2)[0],np.split(fit2.x,2)[1]/np.sum(np.split(fit2.x,2)[1]), c='b', label=r'Second fit')
ax0.plot(x,h)
leg = ax0.legend()
ax0.set_xscale('log')
ax0.set_yscale('log')

ax1 = fig3.add_subplot(132)

ax1.set_title("Check results of the fit")
ax1.set_xlabel(r'$t/\tau_c$')
ax1.set_ylabel(r'log residuals')

ax1.plot(x,Gt_MMM_vec(time=x, params=np.append(lambdaArrInit, gArrInit)), c='k', label=r'Initial guess')
ax1.plot(x,Gt_MMM_vec(time=x, params=fit.x), c='r', label=r'First fit')
ax1.plot(x,Gt_MMM_vec(time=x, params=fit2.x), c='b', label=r'Second fit')
ax1.plot(x,y, c='g', label=r'$f_d(t)$')

leg = ax1.legend()
ax1.set_xscale('log')
ax1.set_yscale('log')

ax2 = fig3.add_subplot(133)

ax2.set_title(r'$f_d(t)$')
ax2.set_xlabel(r'$t/\tau_c$')
ax2.set_ylabel(r'$f_d(t)$')

ax2.plot(x,Gt_MMM_vec(time=x, params=np.append(lambdaArrInit, gArrInit)), c='k', label=r'Initial guess')
ax2.plot(x,Gt_MMM_vec(time=x, params=fit.x), c='r', label=r'First fit')
ax2.plot(x,Gt_MMM_vec(time=x, params=fit2.x), c='b', label=r'Second fit')
ax2.plot(x,y, c='g', label=r'Simulation data')
leg = ax2.legend()
ax2.set_xscale('log')

plt.show()

### Relaxation spectrum $h\left(\tau\right)$

In [ ]:
li=np.split(fit2.x,2)[0]
gi=np.split(fit2.x,2)[1]/np.sum(np.split(fit2.x,2)[1])
list=zip(li,gi)
list.sort()
li=zip(*list)[0]
gi=zip(*list)[1]

In [ ]:
fig8 = plt.figure(figsize=(8, 6))

ax1 = fig8.add_subplot(111)

ax1.set_title("Multimode $h(\\tau)$ fitting")
ax1.set_xlabel(r'$\lambda$')
ax1.set_ylabel(r'$g^\prime$')

ax1.scatter(li,gi, c='k')

leg = ax1.legend()
ax1.set_xscale('log')
ax1.set_yscale('log')

plt.show()

** Shape of the discrete equilibrium spectrum is similar to two linear pieces. We may try to identify them using Ramer-Douglas-Peucker algorithm **

In [ ]:
from math import sqrt

In [ ]:
def distance(a, b):
    return  sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

def point_line_distance(point, start, end):
    if (start == end):
        return distance(point, start)
    else:
        n = abs(
            (end[0] - start[0]) * (start[1] - point[1]) - (start[0] - point[0]) * (end[1] - start[1])
        )
        d = sqrt(
            (end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2
        )
        return n / d

** Find the farthest point from the line connecting first and last point in the spectrum **

In [ ]:
dmax = 0.0
index = 0
for i in range(1, len(li) - 1):
    d = point_line_distance((np.log(li[i]),np.log(gi[i])), (np.log(li[0]),np.log(gi[0])), (np.log(li[-1]),np.log(gi[-1])))
    if d > dmax:
        index = i
        dmax = d
index
#(np.log(li[index]),gi[index])

** Define bilinear BSW spectrum **

In [ ]:
def biliniear_spectrum(t, params):
    if (t > params[1] and t < params[2]):
        return params[0] * t**params[4]
    elif (t > params[2] and t < params[3]):
        return params[0] * params[2]**params[4] * (t/params[2])**params[5]
    else:
        return 0.0
    
def bsw_vs(l, a):
    return (a[0]*( np.exp(l[0])*(l[0]-1) + np.exp(l[1]) - l[1]*np.exp(l[2]) ) - a[1]*( np.exp(l[1])*(l[1]-1) - np.exp(l[2])*(l[2]-1) ))/(np.exp(l[2])-np.exp(l[0]))

** Initial guess from multimode discrete spectrum **

In [ ]:
lambdaBSW = [li[0], li[index], li[-1]]
alphaBSW = [(np.log(gi[index])-np.log(gi[0]))/(np.log(li[index])-np.log(li[0])),(np.log(gi[-1])-np.log(gi[index]))/(np.log(li[-1])-np.log(li[index]))]

bilinear_spectrum_vec=np.vectorize(biliniear_spectrum, excluded=['params'])

fig9 = plt.figure(figsize=(8, 6))

ax0 = fig9.add_subplot(111)

lArray=10**(-3+(np.array(range(1001), float)/1000)*10)
ax0.plot(x,h, c='b')
ax0.plot(lArray, h_BSW(lArray, alphaBSW, lambdaBSW, 2) )
ax0.scatter(li,gi, c='r', label=r'Best SciPy fit')
ax0.set_xscale('log')
ax0.set_yscale('log')
plt.show()

In [ ]:
alphaBSW

## 4. Use initial guess of BSW to fit G(t) with BSW expressions

In [ ]:
fig10 = plt.figure(figsize=(8, 6))

ax1 = fig10.add_subplot(111)

ax1.set_title("Entanglement lifetime distribution")
ax1.set_xlabel(r'$t/\tau_c$')
ax1.set_ylabel(r'$G(t)$')
plt.xlim(xmin=1, xmax=1e7)
ax1.plot(x,y, c='r', label=r'$G(t)$')
ax1.plot(x,Gt_BSW(x, alphaBSW, lambdaBSW, 2, GN0), c='k', label=r'initial BSW')
leg = ax1.legend()
ax1.set_xscale('log')
#ax1.set_yscale('log')

plt.show()

In [ ]:
np.array(Gt_BSW(x, alphaBSW, lambdaBSW, 2,GN0)-y,dtype='float')

In [ ]:
def residuals_Gt_BSW(params):
    n=2
    alpha = params[0:n]
    tauv = params[n:2*n+1]

    return np.array(Gt_BSW(x, alpha, tauv, 2, GN0)-y, dtype='float')

In [ ]:
initial_bsw=np.concatenate((alphaBSW,lambdaBSW))
residuals_Gt_BSW(initial_bsw)

In [ ]:
fit3 = least_squares(residuals_Gt_BSW, initial_bsw, bounds=([-np.inf,0,0,0,0],[0,np.inf,np.inf,np.inf,np.inf]))

In [ ]:
fit3.x

In [ ]:
initial_bsw

In [ ]:
fig11 = plt.figure(figsize=(8, 6))

ax0 = fig11.add_subplot(111)

lArray=10**(-3+(np.array(range(1001), float)/1000)*10)
ax0.plot(x,h, c='b')
ax0.plot(lArray, h_BSW(lArray, alphaBSW, lambdaBSW, 2) )
ax0.plot(lArray, h_BSW(lArray, fit3.x[0:2], fit3.x[2:5], 2) )
ax0.scatter(li,gi, c='r', label=r'Best SciPy fit')
ax0.set_xscale('log')
ax0.set_yscale('log')
plt.show()

In [ ]:
fig12 = plt.figure(figsize=(8, 6))

ax1 = fig12.add_subplot(111)

ax1.set_title("Entanglement lifetime distribution")
ax1.set_xlabel(r'$t/\tau_c$')
ax1.set_ylabel(r'$G(t)$')
plt.xlim(xmin=1, xmax=1e7)
ax1.plot(x,y, c='r', label=r'$G(t)$')
ax1.plot(x,Gt_BSW(x, alphaBSW, lambdaBSW, 2, GN0), c='k', label=r'Initial BSW')
ax1.plot(x,Gt_BSW(x, fit3.x[0:2], fit3.x[2:5], 2, GN0), c='b', label=r'First fit BSW')
leg = ax1.legend()
ax1.set_xscale('log')
ax1.set_yscale('log')

plt.show()

## 5. Improve BSW fit using log-residuals

In [ ]:
def log_residuals_Gt_BSW(params):
    n=2
    alpha = params[0:n]
    tauv = params[n:2*n+1]

    return np.log(np.array(Gt_BSW(x, alpha, tauv, 2, GN0),dtype='float'))-np.log(y)

In [ ]:
fit4 = least_squares(log_residuals_Gt_BSW, fit3.x, bounds=([-np.inf,0,0,0,0],[0,np.inf,np.inf,np.inf,np.inf]))

In [ ]:
fit4.x

In [ ]:
fig13 = plt.figure(figsize=(8, 6))

ax0 = fig13.add_subplot(111)

lArray=10**(-3+(np.array(range(1001), float)/1000)*10)
ax0.plot(x,h, c='b')
ax0.plot(lArray, h_BSW(lArray, alphaBSW, lambdaBSW, 2) )
ax0.plot(lArray, h_BSW(lArray, fit3.x[0:2], fit3.x[2:5], 2) )
ax0.plot(lArray, h_BSW(lArray, fit4.x[0:2], fit4.x[2:5], 2) )
ax0.scatter(li,gi, c='r', label=r'Best SciPy fit')
ax0.set_xscale('log')
ax0.set_yscale('log')
plt.show()

In [ ]:
fig14 = plt.figure(figsize=(8, 6))

ax1 = fig14.add_subplot(111)

ax1.set_title("Entanglement lifetime distribution")
ax1.set_xlabel(r'$t/\tau_c$')
ax1.set_ylabel(r'$G(t)$')
plt.xlim(xmin=1, xmax=2e7)
ax1.plot(x,y, c='r', label=r'$G(t)$')
ax1.plot(x,Gt_BSW(x, alphaBSW, lambdaBSW, 2, GN0), c='k', label=r'Initial BSW')
ax1.plot(x,Gt_BSW(x, fit3.x[0:2], fit3.x[2:5], 2, GN0), c='b', label=r'First fit BSW')
ax1.plot(x,Gt_BSW(x, fit4.x[0:2], fit4.x[2:5], 2, GN0), c='g', label=r'Second fit BSW')
leg = ax1.legend()
ax1.set_xscale('log')
ax1.set_yscale('log')

plt.show()